O conjunto de dados Cora é comumente usado em tarefas de classificação de documentos. Ele é composto por citações científicas e cada documento é representado como uma bag-of-words binária. O desafio com Cora é que ele não é um modelo de embeddings de palavras, como o GloVe ou o FastText. Em vez disso, é um conjunto de dados de documentos.

Se você deseja usar o conjunto de dados Cora para criar embeddings, você terá que treinar um modelo nele. Uma abordagem popular para isso é usar o método Graph Neural Networks (GNNs) ou Graph Convolutional Networks (GCNs) para aprender representações vetoriais dos documentos.

Passos principais:

    Carregar o conjunto de dados Cora: Isso envolve carregar os documentos e o gráfico de citação.

    Treinar um GCN: Usando o gráfico de citação e os recursos dos documentos (bag-of-words binária), você pode treinar um GCN para aprender embeddings para cada documento.

    Gerar Embeddings para Títulos: Depois de treinar o GCN, você pode usá-lo para gerar embeddings para qualquer documento no conjunto de dados Cora.

Para implementar um GCN com PyTorch, você provavelmente usaria uma biblioteca como o PyTorch Geometric. Treinar um GCN é um processo mais envolvido do que simplesmente carregar embeddings pré-treinados como o GloVe.

In [1]:
!python -m venv pytorch
!pytorch\Scripts\activate

# Instalando o PyTorch Geometric

In [8]:
# pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
# !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
# !pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
# !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html

# !pip install torch-geometric

In [14]:
import sys
sys.path.append('C:/Users/marco/fioce/source')

import check_installation
check_installation.check_installation()

PyTorch version: 2.0.1+cu118
PyTorch Geometric version: 2.3.1


In [17]:
import gpu_check

device = gpu_check()

ModuleNotFoundError: No module named 'gpu_check'

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv

# Carregando o conjunto de dados Cora
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=T.NormalizeFeatures())

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

def get_vector(title):
    # Aqui, o processo é mais complicado, pois você precisa converter seu título em um formato aceitável pelo modelo
    # e depois passá-lo pelo GCN para obter um embedding. 
    pass